#下載檔案和引入函式庫

In [1]:
# stupid robot
!wget wget -O "Stupid Question.json" "https://www.dropbox.com/s/jpuh6zzaz3yzegz/Stupid%20Question.json?dl=1"
!apt install chromium-chromedriver
!pip install selenium
# !git clone https://github.com/UDICatNCHU/QAbot_Scoring.git

import time
from time import sleep
import re
from bs4 import BeautifulSoup as bs
import json
# with open('QAbot_Scoring/questions_example.json',encoding="utf-8") as f:
#     qa=json.load(f)

with open('Stupid Question.json',encoding="utf-8") as f:
    qa=json.load(f)

#初始化瀏覽器
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome(options=options)

--2020-12-23 07:44:28--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2020-12-23 07:44:28--  https://www.dropbox.com/s/jpuh6zzaz3yzegz/Stupid%20Question.json?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/jpuh6zzaz3yzegz/Stupid%20Question.json [following]
--2020-12-23 07:44:28--  https://www.dropbox.com/s/dl/jpuh6zzaz3yzegz/Stupid%20Question.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca37107c3e7c5774cfb363b35a8.dl.dropboxusercontent.com/cd/0/get/BFlM1pCJGtUVYrCX_g-n61fiGI6Tb35cXWLhn3pxp_sEi1xkdRun_LVLpvH7hO3C3u8thgxJEOUp0f5lMWlRe2rX0rDca1pKuzS8jQW4ZQ9HM8FmN2WTg32Gt_2h7mAKt3g/file?dl=1# [following]
--2020-12-23 07:44:28--  https://u

#要用的function

In [27]:
stop_word=['出自於','有一','一種','請問','下列','何者','哪者','何人','為什麼','也是','位於','哪些','哪種','哪位','哪個','一個','哪裡','誰是','什麼','之一']

def check(q,txt):
    a=None
    # 抓詞的位置，越往前(數字越小)越有可能是答案
    idxA=txt.find(q['A'])
    idxB=txt.find(q['B'])
    idxC=txt.find(q['C'])
    if idxA!=-1 or idxB!=-1 or idxC!=-1:
        print()
        print('position index:')
        print(q['A'],idxA)
        print(q['B'],idxB)
        print(q['C'],idxC)

    idx=1000000000
    if idxA>-1:
        idx=idxA
        a="A"
    if idxB>-1 and idxB<idx:
        idx=idxB
        a="B"

    if idxC>-1 and idxC<idx:
        idx=idxC
        a="C"

    return a

def getAns(qa):
    ans=None
    title=None
    url=None
    print(qa['Question'])
    
    s=' '.join(re.split('，|。|、|\?',qa['Question'])[:2])
    for sw in stop_word:
        s=s.replace(sw,'')

    print('搜尋',s)
    
    print('A',qa['A'])
    print('B',qa['B'])
    print('C',qa['C'])

    wd.get("https://www.google.com/search?q={}&lr=lang_zh-TW".format(s))

    htmltext = wd.page_source
    soup=bs(htmltext)

    blocks=soup.find_all('div',class_='g')
    if blocks:
        for b in blocks:
            title=b.find('h3',class_='LC20lb DKV0Md')
            if title:
                print(title.text)
                ans=check(qa,title.text) #先check title
                if ans: return ans

                describe=b.find('span',class_='aCOpRe')
                if describe:
                    ans=check(qa,describe.text) #再check 敘述
                    if ans: return ans

                url=b.find('div',class_='yuRUbf').find('a').get('href')

                if url and "wikipedia.org" in url: #如果是wiki的就點進去
                    #改成台灣的wiki
                    url=url.split('/')
                    url[3]='zh-tw'
                    # url[3]='zh-tw'
                    url='/'.join(url)
                    print(url)
                    wd.get(url)
                    soup=bs(wd.page_source)
                    bodyContent=soup.find(id='mw-content-text')
                    if bodyContent:
                        ans=check(qa,bodyContent.text)
                    else:
                        ans=check(qa,wd.page_source)

                    if ans: return ans
        return None
    
    print('被ban了 or 無搜尋結果')
    return None


#start QA

In [3]:
#初始化
Ans=[None]*200 

In [33]:
wd = webdriver.Chrome(options=options) # 瀏覽器初始化
sta=time.time()
for  id in range(0,200): #如果被ban就改range重新執行，瀏覽器初始化後就沒事了
    print(id)
    if id % 20 ==0: wd = webdriver.Chrome(options=options) #每20題初始化一次
    a=getAns(qa[id])
    if a:
        Ans[id]=a
        print('ans',a,qa[id][a])
    else:
        print("no ans")
    print()

print(time.time()-sta)

0
電腦科學名詞，當兩個以上的運算單元，雙方都在等待對方停止執行，以取得系統資源，但是沒有一方提前登出時，就稱為?
搜尋 電腦科學名詞 當兩個以上的運算單元
A 頻寬
B 類比
C 死結
死鎖- 維基百科，自由的百科全書 - Wikipedia
https://zh.wikipedia.org/zh-tw/%E6%AD%BB%E9%94%81

position index:
頻寬 -1
類比 -1
死結 349
ans C 死結

1
臺灣第一名模，是臺灣女藝人，身兼模特兒、主持人、演員、歌手等身分。出生於臺北市，畢業於加拿大多倫多大學西方美術史、經濟學雙主修學士學位。，請問其名為何?
搜尋 臺灣第一名模 是臺灣女藝人
A 林志玲
B 林志達
C 林志揚
林志玲- 维基百科，自由的百科全书

position index:
林志玲 0
林志達 -1
林志揚 -1
ans A 林志玲

2
哪一個宗教起源於古印度河流域迦毗羅衛國(今尼泊爾和印度邊界間靠近尼泊爾)?
搜尋 哪宗教起源於古印度河流域迦毗羅衛國(今尼泊爾和印度邊界間靠近尼泊爾) 
A 基督教
B 佛教
C 伊斯蘭教
佛教- 维基百科，自由的百科全书

position index:
基督教 -1
佛教 0
伊斯蘭教 -1
ans B 佛教

3
主角名為哈克,在雪山迷路，記憶盡失、遇到襲擊時被久遠所救的青年。本作的主人公。年齡約二十餘歲。與其他有著獸耳、獸尾的亞人相對、哈克是個人類。因為連自己的名字也想不起、久遠給了他「ハク」這個名字，這個名字是從久遠故鄉傳說「被傳頌之人」的名字而來的。與周圍相比體能極低、連小孩能簡單做到的事也能讓他苦戰，再加上是懶惰者、用著各種各樣的藉口想在工作上偷懶。但是相對洞察力很優秀、頭腦也轉得很快、能下達大膽的作戰與正確的判斷而多次絕處逢生。是一款由日本AQUAPLUS公司在2015年9月24日發售的冒險+戰略角色扮演遊戲
搜尋 主角名為哈克,在雪山迷路 記憶盡失
A 揭穿你的謊言
B 假面的誘惑
C 受讚頌者 虛偽的假面
《新瑪奇》攜手《受讚頌者虛偽的假面》 - GNN新聞 - 巴哈姆特
《新瑪奇》攜手《受讚頌者虛偽的假面》於17 日推出限時改版 ...
跨界冒險展開！《新瑪奇》攜手日本人氣動漫「受 ... - 麗台運動報
跨界冒險展開！《新瑪奇》攜手日本人氣動漫「受 ...

In [5]:
human=['C', 'A', 'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'A', 'C', 'C', 'C', 'C', 'C', 'C', 'A', 'A', 'B', 'C', 'B', 'C', 'B', 'C', 'C', 'C', 'A', 'A', 'C', 'B', 'C', 'C', 'C', 'C', 'A', 'C', 'A', 'C', 'A', 'B', 'C', 'C', 'B', 'B', 'A', 'A', 'A', 'A', 'B', 'A', 'C', 'A', 'C', 'A', 'C', 'A', 'B', 'B', 'C', 'A', 'C', 'A', 'B', 'B', 'C', 'A', 'A', 'C', 'C', 'C', 'A', 'B', 'B', 'A', 'A', 'B', 'A', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'A', 'B', 'C', 'B', 'C', 'C', 'A', 'C', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'A', 'B', 'A', 'B', 'A', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'B', 'B', 'B', 'C', 'C', 'B', 'A', 'A', 'B', 'C', 'A', 'A', 'C', 'B', 'B', 'C', 'C', 'C', 'B', 'A', 'A', 'B', 'B', 'A', 'A', 'A', 'C', 'C', 'C', 'B', 'C', 'A', 'C', 'B', 'A', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'B', 'A', 'B', 'B', 'A', 'A', 'A', 'C', 'C', 'B', 'B', 'B', 'A', 'B', 'B', 'A', 'B', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'C']

In [34]:
miss=0
noAns=0
for id in range(0,200):
    if Ans[id] != human[id]:
        if Ans[id]: miss+=1
        else: noAns+=1
        print(id)
        print(qa[id]['Question'])
        print('A',qa[id]['A'])
        print('B',qa[id]['B'])
        print('C',qa[id]['C'])
        print('my ans',Ans[id])
        print('human ans',human[id])
        print()

print('miss:',miss)
print('noAns',noAns)

3
主角名為哈克,在雪山迷路，記憶盡失、遇到襲擊時被久遠所救的青年。本作的主人公。年齡約二十餘歲。與其他有著獸耳、獸尾的亞人相對、哈克是個人類。因為連自己的名字也想不起、久遠給了他「ハク」這個名字，這個名字是從久遠故鄉傳說「被傳頌之人」的名字而來的。與周圍相比體能極低、連小孩能簡單做到的事也能讓他苦戰，再加上是懶惰者、用著各種各樣的藉口想在工作上偷懶。但是相對洞察力很優秀、頭腦也轉得很快、能下達大膽的作戰與正確的判斷而多次絕處逢生。是一款由日本AQUAPLUS公司在2015年9月24日發售的冒險+戰略角色扮演遊戲
A 揭穿你的謊言
B 假面的誘惑
C 受讚頌者 虛偽的假面
my ans None
human ans C

6
下列何者是全球人口最多的國家？
A 美國
B 印度
C 中國
my ans A
human ans C

8
超級瑪利歐是任天堂的代表遊戲，下列何者也是?
A 光碟機
B 薩爾達傳說
C 遊戲機
my ans C
human ans B

14
請問下列何人不是中華民國2020總統候選人?
A 蔡英文
B 韓國瑜
C 郭台銘
my ans B
human ans C

59
何者為台灣第一座中華民國建立的國家公園?
A 墾丁國家公園
B 台江國家公園
C 壽山國家自然公園
my ans A
human ans C

68
是臺灣第五座國家公園，可溯至日治時期成立之次高太魯閣國立公園。公園內高山林立，峻秀壯觀，並有稀有的台灣寬尾鳳蝶，及瀕臨絕種的櫻花鉤吻鮭，是臺灣登山路線最多、最複雜的國家公園。
A 玉山國家公園
B 太魯閣國家公園
C 雪霸國家公園
my ans B
human ans C

70
位於臺灣嘉義縣阿里山鄉中山村、南投縣信義鄉東埔村與高雄市桃源區梅山里交界處，海拔3,952.43±0.045公尺，為台灣最高峰，百岳之首。，請問上述為何?
A 聖母峰
B 額菲爾士峰
C 玉山頂峰
my ans None
human ans C

90
教育召集新制將改為一年一訓、一次14天，預計2022年上路。而國防部為滿足戰力需求，研議將教召選充年限，從原本的退伍後8年內，延長為幾年?
A 10
B 12
C 15
my ans B
human ans C

92
第13屆臺北市議員邱威傑，以呱吉為名經營個人頻道，因為在網路上常與「民進黨」支持者

# 將沒答案的填回去

In [ ]:
for id in range(0,200):
    if Ans[id] == None:
        print(id)
        print(qa[id]['Question'])
        print('A',qa[id]['A'])
        print('B',qa[id]['B'])
        print('C',qa[id]['C'])

        a=input()
        Ans[id]=a
print(noAns)

3
主角名為哈克,在雪山迷路，記憶盡失、遇到襲擊時被久遠所救的青年。本作的主人公。年齡約二十餘歲。與其他有著獸耳、獸尾的亞人相對、哈克是個人類。因為連自己的名字也想不起、久遠給了他「ハク」這個名字，這個名字是從久遠故鄉傳說「被傳頌之人」的名字而來的。與周圍相比體能極低、連小孩能簡單做到的事也能讓他苦戰，再加上是懶惰者、用著各種各樣的藉口想在工作上偷懶。但是相對洞察力很優秀、頭腦也轉得很快、能下達大膽的作戰與正確的判斷而多次絕處逢生。是一款由日本AQUAPLUS公司在2015年9月24日發售的冒險+戰略角色扮演遊戲
A 揭穿你的謊言
B 假面的誘惑
C 受讚頌者 虛偽的假面
70
位於臺灣嘉義縣阿里山鄉中山村、南投縣信義鄉東埔村與高雄市桃源區梅山里交界處，海拔3,952.43±0.045公尺，為台灣最高峰，百岳之首。，請問上述為何?
A 聖母峰
B 額菲爾士峰
C 玉山頂峰
92
第13屆臺北市議員邱威傑，以呱吉為名經營個人頻道，因為在網路上常與「民進黨」支持者筆戰，被網友戲稱為何者?
A 綠蟑螂
B 台派
C 中共同路人
105
在俄國支持下，下列哪種主義企圖團結斯拉夫族群的各種理念及作為，成為巴爾幹半島主流思想?
A 達爾文主義
B 泛斯拉夫主義
C 基督教無政府主義
112
下列何者為巴洛克建築?
A 泰姬瑪哈陵
B 聖索菲亞大教堂
C 薩爾茨堡主教座堂
5


# 將有錯的改寫

In [ ]:
while True:
    id=input('要改寫第幾題? ')
    if id=='break': break #打break結束
    id=int(id)
    print(qa[id]['Question'])
    print('A',qa[id]['A'])
    print('B',qa[id]['B'])
    print('C',qa[id]['C'])

    a=input()
    
    Ans[id]=a


要改寫第幾題? break


In [ ]:
print(Ans)

['C', 'A', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'A', 'A', 'C', 'C', 'C', 'B', 'C', 'C', 'A', 'A', 'B', 'C', 'B', 'C', 'B', 'C', 'C', 'C', 'A', 'A', 'C', 'B', 'C', 'C', 'C', 'C', 'A', 'C', 'A', 'C', 'A', 'B', 'C', 'C', 'B', 'B', 'A', 'A', 'A', 'A', 'B', 'A', 'C', 'A', 'C', 'A', 'C', 'A', 'B', 'B', 'A', 'A', 'C', 'A', 'B', 'B', 'C', 'A', 'A', 'C', 'C', 'C', 'A', 'B', 'B', 'C', 'A', 'B', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'A', 'A', 'B', 'C', 'B', 'C', 'B', 'A', 'C', 'A', 'B', 'B', 'C', 'C', 'B', 'B', 'B', 'C', 'A', 'B', 'A', 'B', 'A', 'B', 'B', 'C', None, 'C', None, None, 'B', 'B', 'B', 'C', 'C', 'B', 'A', 'A', None, 'C', 'A', 'A', 'C', 'B', 'B', 'C', 'C', 'C', 'B', None, 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'A', 'C', 'B', 'C', 'A', 'C', 'B', 'A', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'B', 'A', 'B', 'B', 'A', 'A', 'A', 'C', 'C', 'B', 'B', 'B', 'A', 'B', 'B', 'A', 'B', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'C', 'B', 'A', 'C', 'A',

# 彙整答案

In [ ]:
# Ans0_65=
# Ans65_130=
# Ans130_200=

In [ ]:
submit=[None]*200

In [ ]:
submit[0:65]=Ans0_65[0:65]
submit[65:130]=Ans65_130[65:130]
submit[130:200]=Ans130_200[130:200]

In [ ]:
print(submit)